In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
208,AFG,Asia,Afghanistan,2020-08-05,36829.0,82.0,1294.0,6.0,946.072,2.106,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
358,ALB,Europe,Albania,2020-08-05,5750.0,130.0,176.0,4.0,1998.054,45.173,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
572,DZA,Africa,Algeria,2020-08-05,32504.0,532.0,1248.0,9.0,741.237,12.132,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-04,France,191444.0,0.0,30319.0,0.0,0.0,0.0
0,2020-08-04,China,88212.0,0.0,4680.0,0.0,0.0,0.0
0,2020-08-04,Italy,248581.0,0.0,35196.0,0.0,0.0,0.0
0,2020-08-04,Spain,297022.0,0.0,28485.0,0.0,0.0,0.0
0,2020-08-04,United States,4800856.0,0.0,156092.0,0.0,0.0,0.0
0,2020-08-04,World,18744653.0,0.0,700727.0,0.0,0.0,0.0
0,2020-08-04,United Kingdom,306571.0,0.0,46278.0,0.0,0.0,0.0
0,2020-08-04,Germany,211822.0,0.0,9169.0,0.0,0.0,0.0
0,2020-08-04,Iran,315658.0,0.0,17617.0,0.0,0.0,0.0
0,2020-08-04,Turkey,234770.0,0.0,5764.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-05,France,191444.0,192334.0,30319.0,30296.0,-890.0,23.0
0,2020-08-05,China,88212.0,88206.0,4680.0,4676.0,6.0,4.0
0,2020-08-05,Italy,248581.0,248419.0,35196.0,35171.0,162.0,25.0
0,2020-08-04,Spain,297022.0,302814.0,28485.0,28498.0,-5792.0,-13.0
0,2020-08-05,United States,4800856.0,4771087.0,156092.0,156806.0,29769.0,-714.0
0,2020-08-05,World,18744653.0,18519579.0,700727.0,700539.0,225074.0,188.0
0,2020-08-05,United Kingdom,306571.0,306293.0,46278.0,46299.0,278.0,-21.0
0,2020-08-05,Germany,211822.0,212022.0,9169.0,9168.0,-200.0,1.0
0,2020-08-05,Iran,315658.0,314786.0,17617.0,17617.0,872.0,0.0
0,2020-08-05,Turkey,234770.0,234934.0,5764.0,5765.0,-164.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")